# 제1장 자본 시장의 기초

이 장에서는 현대 금융 시장의 핵심 개념과 이것이 QuantConnect에서 어떻게 표현되는지 소개합니다. 현대 미국 시장, 데이터 피드, 그리고 이후 장에서 사용될 자산 클래스에 대해 다룰 것입니다. QuantConnect에 익숙한 독자들은 이 장을 건너뛰어도 됩니다.

---

## 시장 메커니즘

미국에는 11개의 주요 증권 거래소가 있습니다. 그중 가장 큰 두 곳은 뉴욕 증권 거래소(NYSE)와 전국 증권 딜러 협회 자동 호가 시스템(NASDAQ)입니다. 이러한 거래소들의 거래는 증권 정보 처리기(SIP)에 의해 단일 데이터 피드로 통합됩니다. 이 피드는 증권거래위원회(SEC)가 미국 공개 시장에 게시된 최적 가격을 보여주는 전국 최고 매수호가 또는 매도호가(NBBO)를 결정하는 데 도움을 줍니다. 100주 이상에 대한 새로운 호가가 더 나은 가격을 제시할 때, 이는 NBBO로 표시됩니다. 100주 미만의 호가나 거래인 '홀수 주문(odd lots)'은 이 가격 산정에서 제외됩니다. 그림 1.1은 이러한 흐름을 보여줍니다.

<img src="./images/fig_01_01.png" width=800>

그림 1.1 소매 및 기관 거래의 공개 및 비공개 시장 간 흐름과 국가 최적 가격의 기원.

중개업체들은 종종 주문을 "시장 외부"에서 실행할 수 있도록 마켓 메이커에게 보냅니다. 이러한 주문을 실행하는 마켓 메이커는 NBBO 가격 범위 내에서 체결을 제공해야 합니다. 또한, 이러한 장외 거래는 거래 보고 시설(TRF)에 보고되며 결국 SIP 데이터 피드에 포함됩니다. 일부 중개업체는 직접 시장 접근(DMA)을 제공하여 귀하의 주문이 특정 거래소로 직접 라우팅될 수 있도록 합니다. 그러나 DMA를 사용하면 항상 해당 자산에 대한 최고의 국가 가격을 얻을 수 있는 것은 아니므로, 이 옵션을 사용할 때는 주의하는 것이 중요합니다.

---

## 시장 참여자

**거래는 "연극"이다**

시장이 극장이라면, 거래는 "연극"입니다. 셰익스피어의 연극처럼, 거래, 특히 알고리즘 거래는 고도로 조율되고 각본이 짜인 활동입니다. 거래를 셰익스피어의 연극에 비유하는 것은 아마도 셰익스피어를 무덤에서 뒤집게 할 것입니다. 하지만 그는 이 문학 애호가의 억지 은유를 용서할 것입니다.

**거래의 무대와 기본 규칙—지정가 주문장**

이 비유를 더 확장하자면, 거래의 무대는 "지정가 주문장(limit order book)"입니다. 이는 증권의 지정가 주문 가격을 열에 나열하는 일종의 원장입니다. 왼쪽에는 매수호가(bid prices), 즉 트레이더들이 증권을 구매할 의사가 있는 가격과 주문의 수량 또는 "크기"(주식의 경우 일반적으로 100주 단위의 배수; 선물 및 옵션의 경우 계약 수)가 있는 열이 있습니다. 이 원장의 반대편에는 매도호가(ask prices), 즉 트레이더들이 증권을 판매할 의사가 있는 가격과 크기가 있는 열이 있습니다. 가격은 일반적으로 위에서 아래로 높은 것부터 낮은 것까지 정렬되며, 최고 매수호가와 매도호가는 중간에서 만납니다. 매수호가는 매도호가보다 높을 수 없습니다. 즉, 매수호가와 매도호가는 교차하지 않습니다. 그렇지 않으면 구매자와 판매자가 한쪽 또는 양쪽에 이익이 되는 방식으로 서로의 주문을 충족할 수 있게 됩니다. 현재 최고 매도호가보다 높은 가격으로 시장에 들어오는 모든 매수 주문은 사실상 시장가 주문이며, 최고 매도호가의 크기까지 매칭된 다음, 다음 최고 매도호가 등으로 매칭되어 전체 매수 주문이 "북 워킹(walk-the-book)"이라는 과정을 통해 채워지거나, 남은 매도호가가 최고 매수호가보다 높을 때까지 진행됩니다. 이 때문에 지정가 주문을 게시하는 트레이더는 일반적으로 최고 매도호가 아래로 매수호가를 게시하고 최고 매수호가 위로 매도호가를 게시합니다.

최고 매수호가와 매도호가 사이에는 스프레드가 있기 때문에, 지정가 주문은 즉시 체결되지 않으며, 실제로 때때로 전혀 체결되지 않습니다. 거래의 즉시성을 보장하기 위해, 충분히 유동성 있는 시장에서는 트레이더가 "시장가 주문"을 게시할 수 있으며, 이는 시장 매도 주문의 경우 최고 매수호가와 매칭되고 시장 매수 주문의 경우 최고 매도호가와 매칭됩니다. 현재 매수호가와 매도호가에 충분한 크기가 없다면, 시장 매도 및 매수 주문은 앞서 설명한 대로 주문장을 워킹합니다: 시장 매수 주문은 점진적으로 더 높은 가격을 지불하게 되고, 반대로 시장 매도 주문은 점진적으로 더 낮은 가격을 수락하게 됩니다. 시장가 주문의 크기가 이용 가능한 지정가 주문의 크기에 비해 크다면, 주문은 주문장에서 더 깊게(즉, 구매자의 경우 더 높게 또는 판매자의 경우 더 낮게) 워킹하여 증권의 거래 가격이 즉시 상승하거나 하락하게 됩니다. 이것은 거래의 불리한 가격 영향의 한 형태입니다.

**배우들—유동성 트레이더, 마켓 메이커, 정보를 가진 트레이더**

이제 무대가 설정되었으니, 배우들, 또는 더 적절하게는 연극의 캐릭터나 역할을 소개하겠습니다. 배우들처럼, 트레이더들도 여러 역할을 수행할 수 있으며, 때로는 같은 연극에서 또는 심지어 동시에 수행하기도 합니다.

**유동성 트레이더**

"유동성 트레이더"는 "펀더멘털 트레이더" 또는 경멸적으로 "노이즈 트레이더"라고도 불리며, 우월한 정보(항상 내부자 정보는 아님)로부터 이익을 얻는 것 이외의 목적으로 포지션에 들어가거나 나오는 것이 주요 목표인 트레이더입니다. 예를 들어, 뮤추얼 펀드 매니저가 벤치마크 지수에 맞추기 위해 주식 포트폴리오를 재조정하기로 결정합니다. 펀드 매니저의 지시에 따라 행동하는 트레이더는 유동성 트레이더입니다. 같은 펀드 매니저가 공개적으로 이용 가능한 정보를 기반으로 거래를 하고 자신의 모자이크 관점에서 이익을 얻기 위해 일부 주식을 사거나 파는 매크로 예측가, 섹터 전문가, 또는 주식 선택자일 수 있습니다. 그래도 그런 펀드 매니저는 다른 사람들이 가지고 있지 않은 우월한 정보를 가지고 있지 않습니다. 그녀의 거래를 실행하는 트레이더는 "펀더멘털 트레이더"라고 존중받을 수 있지만, 근본적으로 그는 유동성 트레이더와 다르지 않습니다. 마지막으로, 거래를 위한 거래를 하는 규율 없는 트레이더들도 있으며, 우리는 그들을 안전하게 "노이즈 트레이더"라고 부를 수 있습니다. 트레이더가 펀더멘털이든 노이즈든, 그들의 목표는 우월한 정보 없이 매수 또는 매도 주문을 완료하는 것입니다.

**마켓 메이커**

유동성 트레이더가 유동성을 원한다면, 누가 그에게 술을 따라줄까요? 음, 그것은 거래의 반대편에 있는 유동성 트레이더일 수 있습니다. 대부분의 유동적인 주식 거래는 거래의 반대편에서 동시에 시장가 주문을 매칭함으로써 발생할 가능성이 높습니다. 그리고 그러한 주문의 대부분은 주식 거래소에 도달하기 전에 브로커 딜러에 의해 매칭됩니다. 이것은 주문을 내부적으로 "교차" 또는 "상쇄"함으로써 이루어지며, 일반적으로 중간 가격, 즉 지정가 주문장에서 최고 매수호가와 매도호가 사이의 중간 지점에서 이루어집니다.

반대편에서 충분한 시장가 주문이 없다면, 시장가 주문은 "매수호가를 치거나" "매도호가를 들어올릴 것"입니다. 앞서 논의한 바와 같이, 최고 매수호가나 매도호가에 충분한 크기가 없으면 주문장을 워킹할 것입니다. 여기서, 다른 지정가 주문들이 유동성을 찾는 시장가 주문을 충족시킬 것입니다. 거래소는 트레이더에게 유동성 제공에 대한 보상으로 체결된 지정가 주문에 리베이트를 지급합니다.

거래를 촉진하고 거래 가격 또는 즉시성을 향상시키는 전략은 일반적으로 "마켓 메이커"가 배치하는 전략입니다. 전통적으로, 거래가 주식 거래소 바닥(예: NYSE)에서 수행될 때, 전담 마켓 메이커는 부스에 물리적으로 위치하고 있었으며 그들의 임무는 할당된 주식 목록에 대한 거래를 매칭하는 것이었습니다. 그들은 등록된 매수 및 매도(지정가) 주문이 미결 주문의 시장을 청산하기에 충분하지 않을 때(예: 최고 매수호가와 매도호가 가격이 너무 멀리 떨어져 있어 시장이 정체될 때) 준비되어 있었습니다. 이러한 상황에서, 마켓 메이커의 임무 및 기회는 최고 매수호가와 매도호가 가격 사이에 들어가는 주문을 게시하여 유동성 트레이더가 그녀의 더 나은 가격에서 거래하도록 장려하는 것이었습니다. 그 대가로, 마켓 메이커는 매수호가와 매도호가 스프레드에서 이익을 얻었습니다. 즉, 그녀는 자신의 매수호가 가격에 구매하고 현재 또는 개선된 매도호가 가격에서 더 높게 판매했습니다. 이전 서술은 과거 시제입니다. 왜냐하면 마켓 메이킹 활동의 대부분이 전자 거래 플랫폼으로 이동했기 때문입니다. 일부 플로어 거래는 여전히 유동성이 적은 주식 및 기타 자산 클래스에 존재합니다. 예를 들어, 시카고 옵션 거래소는 여전히 주식 및 주식 지수 옵션을 거래하는 피트를 운영하고 있지만, 시카고 상품 거래소는 2020년 3월 COVID-19로 인해 피트 거래를 중단하고 이 폐쇄가 영구적일 것이라고 결정했습니다.

현대 전자 거래 시대에서, 마켓 메이커는 이제 거의 독점적으로 알고리즘 및 고빈도 거래 프로그램입니다. 여전히 특정 주식에서 시장을 만드는 것이 주요하거나 독점적인 일인 트레이더가 있지만, 누구나 유동성 트레이더를 위한 결과—더 나은 가격 및 즉시성—를 향상시키는 전략을 배치함으로써 의도적으로 또는 의도하지 않게 시장을 만들 수 있습니다.

**정보를 가진 트레이더**

잠깐, 마켓 메이커는 유동성 트레이더만 돕나요? 다른 유형의 트레이더는 어떻습니까? 마켓 메이커는 모든 사람을 돕지 않나요? 네, 마켓 메이킹 활동은 서로 경쟁하는 다른 마켓 메이커를 제외한 모든 사람을 돕습니다. 그럼에도 불구하고, 마켓 메이커는 다른 마켓 메이커보다 "정보를 가진 트레이더"를 더 경계합니다. 내부자 거래에 관한 논쟁을 피하기 위해, 우리는 정보를 가진 트레이더를 소수의 에이전트만 알고 있고 대중에게 공개되지 않은 우월하거나, 특권적인 정보를 보유한 사람으로 광범위하게 정의할 것입니다. 앞서 논의했듯이, 트레이더나 펀드 매니저는 금융 시장과 개별 증권의 미래 결과에 대한 독특한 의견을 가질 수 있습니다. 그리고 그녀는 우수한 예측 또는 분석 기술을 가질 수 있습니다. 그러나 그녀가 다른 사람들이 가지고 있지 않은 우월한 정보를 가지고 있지 않다면, 그녀는 정보를 가진 트레이더로 간주되지 않습니다. 반면에, 우월한 정보가 항상 금융 증권의 기본 가치에 관한 것일 필요는 없습니다. 예를 들어, 브로커가 특정 펀드 매니저가 시장 가격에 상당한 영향을 미칠 큰 블록 거래를 하려고 한다는 정보를 가지고 있다고 가정해 보겠습니다. 이 정보를 가진 트레이더는 펀드 매니저보다 앞서 거래하는 프론트러닝이라는 불법적인 계획으로 이익을 얻을 수 있습니다.

내부자 거래 또는 특권 정보에 기반한 거래의 이점이나 해악을 논하는 것은 이 저자의 일이 아닙니다. 요점은 정보를 가진 트레이더는 다른 트레이더들보다 미래 결과에 대해 훨씬 더 확실하게 알고 있기 때문에 다른 트레이더들보다 큰 이점을 가지고 있다는 것입니다. 이러한 정보가 없는 트레이더에는 유동성 트레이더와 마켓 메이커가 모두 포함됩니다. 고빈도 거래 장비를 갖춘 마켓 메이커조차도 저기술 정보를 가진 트레이더를 이길 수 없습니다. 상대방이 정보를 가진 트레이더인지 알기는 매우 어렵습니다. 이는 마켓 메이커가 정보를 가진 트레이더와의 거래 위험을 통합하기 위해 자신의 전략을 조정해야 함을 의미합니다.

**AI 배우 구함!**

기계학습과 인공지능(AI)의 흥미로운 약속 중 하나는 정보를 가진 트레이더의 활동 패턴을 감지하도록 알고리즘을 훈련시킬 수 있다는 것입니다. 시장에서 매일 생성되는 테라바이트의 거래 데이터를 선별하고, 거래 가능한 패턴을 분석하고 인식하는 것은 더 이상 인간적으로 가능하지 않습니다. 인간이 패턴을 인식할 수 있다 하더라도, 그는 이에 반응하기에 충분히 빠르지 않을 것입니다. 트레이더들이 월스트리트 저널의 주식 표나 인베스터스 비즈니스 데일리의 작은 썸네일 크기의 가격 차트를 연구함으로써 주식 선택 기술을 연마하고 거래 기회를 찾을 수 있었던 시대는 지났습니다. 따라서, 거래 가능한 신호를 감지하고 행동하기 위해 충분히 빠르게 시장 데이터를 처리하고 분석하기 위해 강력한 기계학습 알고리즘을 실행하는 빠른 컴퓨터에 의존하는 것이 중요해졌습니다.

---

## 데이터와 데이터 피드

시장 참여자들이 매수호가, 매도호가를 입력하거나 거래를 실행할 때, 시장을 통해 방송되는 이벤트가 생성됩니다. 수백만 명의 트레이더와 투자자들로 인해, 거래일 동안 이벤트의 지속적인 흐름과 엄청난 양의 데이터가 생성됩니다. 흥미롭게도, 이 흐름은 점심 시간 무렵에 줄어들었다가 트레이더들이 시장 마감을 위해 접속하는 오후 3시경에 다시 증가하는 경향이 있습니다. 시장 개장과 마감 시 거래량은 일중 거래량의 2~3배에 달할 수 있습니다.

1960년대까지, 이러한 거래와 호가의 흐름은 후대를 위해 종이 테이프 릴에 기록되어 티커 테이프를 만들었습니다. "테이프를 다시 재생한다"는 말은 이러한 흐름이 순서대로 나타나는 것을 반복하는 것에서 유래했습니다.

```python
self.time \# 백테스팅 및 실시간 거래에서의 시간
```

데이터 스트림이 잠재적인 오류나 불완전성에도 불구하고 나타나는 그대로 기록된다면, 그것은 특정 시점 데이터셋이 됩니다. 특정 시점 데이터는 캡처된 정확한 순간의 데이터에 대한 원시 표현이지만, 피드 연결이나 거래소 자체의 오류를 포함할 수 있습니다.

이의 전형적인 정기적인 예는 실행된 거래의 지연 보고입니다. 거래소는 참가자들에게 90초 이내에 거래를 보고하도록 요구하지만, 일부는 그보다 훨씬 늦게, 그날 초기에 발생한 거래의 가격으로 보고할 수 있습니다. 이는 가격 불연속성처럼 보이며, 이를 필터링하는 데 주의하지 않는 신규 퀀트 트레이더들에게는 꽤 당혹스러울 수 있습니다.

최근에는 일부 급격한 시장 하락이 알고리즘 거래 프로그램에 의해 악화되었습니다. 가장 유명한 것은 2010년 플래시 크래시로, 2010년 5월 6일 오후 2시 30분경에 시작하여 약 35분간 지속되었습니다(그림 1.2 참조). 2010년 플래시 크래시 이후, 증권 거래소들은 새로운 거래 제한 또는 서킷 브레이커를 발표했습니다. NYSE와 NASDAQ이 주도하는 몇몇 거래소들은 충돌 기간 동안 매우 낮은 가격에 이루어진 거래를 소급하여 취소하기 시작했습니다. 이러한 취소는 거래 후 몇 분에서 몇 시간 내에 이루어지며, 사전에 알 수 없습니다.

<img src="./images/fig_01_02.png" width=800>

그림 1.2 2010년 5월 6일 시장 폭락 동안 다양한 자산의 상대적 성과.

시장 데이터 피드는 세 가지 주요 유형의 틱 데이터를 생성합니다:
거래 틱-주문 체결에 대한 판매 보고서로, 거래소 장소 및 지연과 같은 주문에 대한 정보를 전달하는 플래그와 함께 제공됩니다.

호가 틱-특정 수량의 주식을 특정 가격에 사거나 팔겠다는 제안. 최고 또는 가장 높은 매수호가 가격(구매 제안)과 최고 또는 가장 낮은 매도호가 가격(판매 제안) 사이의 차이가 매수-매도 스프레드를 형성합니다. 미국 시장의 경우, 국내 최고 호가는 전국 최고 매수호가 또는 매도호가(NBBO)를 형성합니다. 이러한 틱은 시장에 현재 최고 가격을 알리기 위한 특별한 플래그와 함께 방송됩니다.

QuantConnect에서, 이 데이터는 각 데이터 이벤트에서 틱 목록으로 전달됩니다. 참고로, "tick_type"은 데이터 포인트가 거래 틱인지 호가 틱인지를 식별하는 속성입니다.

```python
def on_data(self, slice):
    ticks = slice.ticks.get(self._symbol, []) # 찾지 못하면 빈 배열
    for tick in ticks:
        price = tick.price # 가격 제한 또는 거래
    trade_or_quote = tick.tick_type  # TickType.TRADE 또는 QUOTE
```

통합 데이터-틱 데이터는 하루에 수십억 개의 이벤트를 생성합니다. 많은 디스크 공간을 소비하고 처리 속도가 느립니다. 데이터셋을 더 작게 만들고 연구하기 쉽게 하기 위해, 틱은 바(bar)로 집계될 수 있습니다. 가장 일반적으로, 집계는 고정된 시간 간격 동안 가격(시가, 고가, 저가, 종가), 거래량, 마지막 매수호가, 마지막 매도호가 등을 나타냅니다. QuantConnect는 시장의 틱 스트림을 거래 및 호가 바로 통합합니다(qnt.co/book-consolidate 참조). 거래 바는 일정 기간 동안의 판매 가격을 나타내고, 호가 바는 바 기간 동안 집계된 매수호가와 매도호가를 나타냅니다(그림 1.3).

```python
def on_data(self, slice):
    trade_bar = slice.bars.get(self._symbol) # 거래 바 가져오기
    quote_bar = slice.quote_bars.get(self._symbol) # 호가 바 가져오기
```

<img src="./images/fig_01_03.png" width=800>

그림 1.3 거래 바 형성 및 속성.
각 호가 바는 고정된 간격 동안 매수호가 및 매도호가 데이터의 집계를 나타내는 하위 매수호가 및 매도호가 바를 속성으로 가집니다. 이들은 또한 시가-고가-저가-종가(OHLC) 바 형태입니다. QuantConnect에서, 자산에 대한 호가 데이터가 사용 가능할 때, 이는 이전 거래 가격을 사용하는 것보다 더 정확한 표현이므로 주문 체결 모델링에 사용됩니다(그림 1.4).

```python
quote_bar.bid.close # 집계된 매수호가 가격 바의 종가
quote_bar.bid.close # 집계된 매수호가 가격 바의 종가
```

<img src="./images/fig_01_04.png" width=800>

그림 1.4 호가 바 형성 및 속성.

**커스텀 및 대체 데이터**

가격 외에도, 자산 가격의 움직임에 대한 인사이트를 제공하는 다양한 다른 데이터 소스가 있습니다. 광범위하게, 이 데이터 카테고리는 대체 데이터라고 불리며 이미징, 부동산, 날씨, 운송, 규제, 그리고 지리적 위치, 리뷰, 감성, 거래를 포함한 고객 추적 제품군을 포함합니다.

QuantConnect는 60개 이상의 데이터셋(qnt.co/book-datasets)을 호스팅하며 정기적으로 새로운 데이터셋을 온보딩합니다. 데이터는 몇 줄의 코드로 전략에 추가할 수 있습니다. 다음 예제는 스트리밍 뉴스 서비스인 TiingoNews에서 Apple에 관한 뉴스 기사를 제공합니다:

```python
self._aapl = self.add_equity("AAPL",
Resolution.MINUTE).symbol
self._dataset_symbol = self.add_data(TiingoNews, self._aapl).symbol
```

대체 데이터에서 거래 가능한 신호를 생성하는 것은 과학이며, 우리는 이후 장에서 이러한 데이터셋을 분석하는 방법에 대해 더 자세히 살펴볼 것입니다. QuantConnect에 필요한 데이터가 없다면, 데이터 형식을 클래스로 정의하고 add_data 메서드를 사용하여 전략에 추가함으로써 커스텀 데이터셋을 업로드할 수 있습니다. 다음 코드 스니펫에서, 우리는 데이터의 속성을 정의하고 소스 CSV 파일을 파싱할 커스텀 클래스인 MyFactorDataset을 전달하고 있습니다:

```python
self._custom_symbol = self.add_data(MyFactorDataset, "Factors").symbol
```

---

## 브로커리지와 거래 비용

브로커는 자본 시장과 고객 사이의 중개자 역할을 합니다. 그들은 고객 자산을 보유하고, 거래소에서 거래를 청산하며, 고객이 마진 한도 내에 머물도록 하여 SEC 및 FINRA 마진 규칙을 시행합니다. 브로커리지는 일반적으로 청산 회사와 함께 거래를 청산하고 결제합니다. 대부분의 기관 및 소매 거래는 브로커리지 회사를 통해 실행됩니다. 이 책에서는 청산 회사의 기능에 대해 논의하지 않을 것입니다. 왜냐하면 그것은 여기 독자들과 관련이 없기 때문입니다.

브로커는 다양한 수수료 구조, 지원되는 자산, 주문 유형(gnt.co/book-order-types)을 갖고 있습니다. 일부 브로커리지는 거래를 채우기 위해 마켓 메이커에게 라우팅하여 특정 마켓 메이커에게 주문 흐름을 유도함으로써 리베이트를 얻습니다. 일부는 다른 고객 주문과 함께 "네팅"이라고 불리는 과정에서 내부적으로 주문을 채울 수 있습니다. 모든 내부적으로 채워진 주문은 공개 시장의 전국 최고 매수-매도 가격 내에서 실행되어야 합니다.

브로커리지는 최적의 체결을 찾는 일부 독점 주문 유형을 포함하여 많은 다양한 유형의 주문을 지원할 수 있습니다. 다른 브로커리지는 주문이 일단 접수된 후에는 업데이트할 수 없는 능력과 같은 미묘한 제한을 가지고 있습니다.

QuantConnect는 12개의 주문 유형을 구현합니다. 다음은 이후 예제에서 사용되는 두 가지 일반적으로 사용되는 주문 유형의 예입니다:

```python
# 거래소 개장 시 개장 경매에서 체결을 시도
self.market_on_open_order(symbol, quantity, tag,order_properties)

# stop_price에 도달하면, 시장가 주문을 제출
self.stop_market_order(symbol, quantity, stop_price, tag, order_properties)
```

참고로, 일부 주문 유형은 브로커리지에서 지원되지 않을 수 있습니다. QuantConnect에서, 모든 주문은 주문이 체결되기 전에 주문을 업데이트하거나 취소하는 데 사용할 수 있는 주문 티켓(gnt.co/book-order-ticket)을 반환합니다. 이를 의류 보관소에서 코트를 찾을 수 있게 해주는 코트 티켓처럼 생각하세요.

<img src="./images/fig_01_05.png" width=800>

그림 1.5 알고리즘과 브로커리지 간의 주문 메시징.

**거래 비용**

거래에는 많은 유형의 거래 비용이 있습니다. 브로커리지(gnt.co/book-fee-model)와 세금에서 오는 수수료와 같은 일부는 명시적이고, 다른 것들은 단순히 거래 행위에서 발생하는 더 암묵적인 비용입니다. 양적 연구를 위해, 이러한 비용이 어디서 오는지 이해하는 것이 중요합니다. 그래야 이를 모델링하고 이러한 비용을 줄이기 위해 거래를 최적화할 수 있습니다.

**거래 수수료**

브로커는 주문당 또는 주식당 수수료를 부과하며, 이는 브로커리지 계정에 직접 청구됩니다. QuantConnect에서는 특정 거래 수수료를 시뮬레이션하기 위해 수수료 모델을 사용하거나, 더 일반적으로 특정 브로커의 제한 및 비용을 시행하는 브로커리지 모델을 사용할 수 있습니다.

```python
# 이 증권에 대해 $1의 커스텀 수수료 모델 설정
security.set_fee_model(ConstantFeeModel(1))
# 브로커의 모든 수수료 및 제한 설정
self.set_brokerage_model(BrokerageName.INTERACTIVE_BROKERS_BROKERAGE)
```

**매수-매도 스프레드**

시장가 주문의 경우, 이러한 암묵적 비용 중 가장 중요한 것은 매수-매도 스프레드입니다: 시장에서 자산의 최고 매수가와 매도가 사이의 차이(일반적으로 앞서 다룬 NBBO에서 추정됨). 시장가 주문이 체결될 때, 그것은 스프레드를 건너 가장 좋은 가용 가격에서 체결됩니다(그림 1.6). 일부 실무자들은 낙관적으로 체결이 마지막 거래 판매 가격에서 발생할 것이라고 가정하는데, 이는 일반적으로 스프레드의 중간 어딘가에 있습니다. 실시간 거래에서, 스프레드는 수익성에 상당한 영향을 미칠 수 있으며 연구에서 고려되어야 합니다.

<img src="./images/fig_01_06.png" width=800>

그림 1.6 스프레드 교차: 시장가 매도 주문은 매수가에서 체결되고, 시장가 매수 주문은 매도가에서 체결됩니다. 마지막 판매는 매수가, 매도가, 또는 그 사이 어딘가에 있을 수 있습니다.

QuantConnect에서, 주문은 이 스프레드를 고려하여 호가 데이터를 사용하여 체결됩니다. 기본 체결 동작이 사용 사례에 맞지 않는 경우, 플러그인으로 동작을 사용자 정의할 수 있습니다.

```python
security.set_fill_model(ImmediateFillModel())
```

**슬리피지**

거래 슬리피지(gnt.co/book-slippage-model)는 예상 실행 가격과 거래의 최종 체결 가격 사이의 차이입니다. 유동성 있는 자산에 대한 소규모 주문의 가장 일반적인 원인은 주문이 체결되는 동안 자산 가격이 변하는 시간입니다(그림 1.7).

<img src="./images/fig_01_07.png" width=800>

그림 1.7 슬리피지 비용은 유동성 있는 자산과 유동성이 없는 자산 사이에 다릅니다.

유동성이 없는 자산이나 대규모 주문의 경우, 슬리피지는 거래 주문 자체의 일시적 또는 영구적 가격 영향에서 올 수 있습니다. 가격 영향의 직접적인 메커니즘은 우리가 앞서 논의한 "북 워킹(walk-the-book)"이라는 과정을 통해 이루어집니다. 주문장은 다중 레벨에서의 매수-매도 가격을 표시된 크기와 결합합니다. 대규모 시장가 매수 주문(즉, 주문장에서 최고 매도가에서 즉시 실행되는 주문)은 먼저 최고 매도가에서 체결된 다음, 전체 주문이 체결될 때까지 순차적으로 점점 더 높은 매도가에서 체결됩니다. 이로 인해 거래 가격은 "북 워킹"에 의해 상승합니다. 시장가 매도 주문은 같은 논리를 따라 가격을 내려가며 북을 워킹합니다. 유동성 있는 시장의 경우, 마켓 메이커가 시장가 주문이 도착하기 전의 원래 가격에 가까운 가격으로 주문장을 보충하기 때문에 가격 영향은 일시적일 수 있습니다. 그러나 마켓 메이커가 정보를 가진 트레이더 활동을 의심하는 경우, 그들은 마지막 거래 가격 근처(또는 더 나쁜)에서 더 높거나 낮은 가격으로 주문장을 보충할 것입니다. 이 경우, 가격 영향은 영구적이 됩니다. 일시적인 가격 영향은 더 작은 단위로 거래함으로써 완화될 수 있지만, 예상되는 영구적인 가격 영향은 거래의 실제 크기를 위장하기 위해 더 정교한 거래 알고리즘을 필요로 할 것입니다.

QuantConnect는 기본적으로 자산이 유동적이라고 가정하지만, 더 미묘한 체결 동작을 사용자 정의하기 위한 여러 대안 모델을 제공합니다:

```python
# 주문장 상단에서 즉시 체결 가정
security.set_slippage_model(NullSlippageModel())

# 체결 시 모델-추정 시장 영향
security.set_slippage_model(MarketImpactSlippageModel(self))
```
---

## 증권 식별자

시간이 지남에 따라 자산을 신뢰성 있게 추적하기 위해서는 자산을 고유하게 식별하는 것이 중요합니다. 주식에서는 회사명 변경, 합병, 상장 폐지와 같은 기업 이벤트로 인해 백테스트가 오해를 불러일으키고 부정확해질 수 있습니다. 수백 개의 주식 포트폴리오를 거래하는 경우 이러한 문제를 발견하기 어려울 수 있습니다. 지난 수십 년 동안 자산 추적을 돕기 위해 많은 고유 자산 식별자가 개발되었습니다:

- CUSIP—미국과 캐나다로 제한된 독점 시스템으로 여러 자산 클래스를 다룹니다. CUSIP은 8자리 영숫자 문자열이거나, 첫 8자리에서 인코딩된 "확인 숫자"를 포함하면 9자리입니다.
- FIGI—블룸버그의 무료 독점 데이터베이스 조회 서비스.
- ISIN—CUSIP을 가진 대부분의 미국 및 캐나다 자산의 경우, ISIN은 국가 코드(예: US, CA)와 8자리 CUSIP의 연결입니다.

퀀트 트레이더는 Algoseek과 같은 이러한 식별자를 포함하는 제3자 자산 가격 데이터셋을 구독하거나, 공급자로부터 직접 식별자 데이터셋을 구독하여 이러한 식별자에 접근할 수 있습니다. 그러나 이러한 식별자를 사용하는 데는 몇 가지 주요 제한 사항이 있습니다:

- 이들은 종종 비싸고 제한적인 라이센스가 필요한 독점 데이터셋입니다.
- 식별자 데이터셋은 자체적으로 완전하지 않을 수 있으며, 회사에 대한 정보를 조회하기 위해 병렬 데이터베이스가 필요할 수 있습니다.
- 다양한 자산 클래스나 국가 시장의 커버리지가 제한적입니다.

QuantConnect는 오픈 소스 인코딩 기술을 구현하여 이러한 제한을 해결합니다. 식별자(qnt.co/book-security-id 참조)는 자산을 "지문화"하는 데 필요한 데이터의 해시로, 완전히 자급자족적이므로 데이터베이스 조회가 필요하지 않습니다. 이 식별자는 "Symbol"이라고 불리며, 255개 국가에서 최대 99개의 자산 클래스를 지원하고 파생 자산 유형을 처리합니다. 라이센스 비용이 없습니다.
2014년, 구글은 의결권이 없는 C클래스 주식의 티커 스왑과 기업공개(IPO)를 수행했습니다(qnt.co/book-goog-ino). 그들은 스왑을 용이하게 하기 위해 GOOCV와 GOOAV를 임시 자산으로 상장한 다음, 이전 A클래스 주식 티커가 C클래스에 사용되도록 티커를 뒤집었습니다(그림 1.8).

<img src="./images/fig_01_08.png" width=800>

그림 1.8 복잡한 티커 변경 이력에 대한 증권 식별.
심볼 객체는 티커 이름 변경에도 불구하고 변경되지 않으며, 표현된 회사에 대한 일관된 연결을 유지합니다. Symbol 객체의 id 속성은 인코딩된 해시를 저장하며, 이는 전송(JavaScript Object Notation [JSON]으로 인코딩) 또는 디스크에 저장하는 데 유용할 수 있습니다(그림 1.9).

<img src="./images/fig_01_09.png" width=800>

그림 1.9 단순한 티커 변경 이력에 대한 증권 식별.
거래에 심볼 객체를 사용하면 티커 이름 변경 및 기업 합병을 통해 참조된 엔티티가 동일하게 유지됩니다. 문자열로 직렬화되면 SPY R735QTJ8XC9X와 같이 보이며 symbol.id 속성에 저장됩니다. 이 두 부분 문자열은 IPO 티커, 증권 유형, 날짜, 옵션 행사가, 권리, 그리고 자산이 상장된 시장을 포함하는 데이터의 base64 인코딩 세트입니다. 시장 속성은 동일한 문자열 값을 가지지만 다른 기초 자산을 나타내는 티커를 구분합니다. 예를 들어, BTCUSD는 코인베이스와 크라켄에 상장되어 있지만, 가격이 다르고 다른 거래소에서 쉽게 판매되지 않기 때문에 다른 장소로 취급됩니다.

---

## 자산과 파생상품

이 책의 목적상, 다음 장은 이후 예제에서 사용되는 다섯 가지 자산 클래스에 초점을 맞추고 있습니다. 알고리즘에 자산을 추가하는 방법과 이후 예제에서 사용되는 데이터에 접근하는 방법을 살펴보겠습니다.

퀀트 모델링은 잘 포맷되고 유지되는 데이터가 존재하는 유동적인 시장에서 가장 쉽습니다. QuantConnect 가격 데이터의 대부분은 Algoseek(algoseek.com)에서 제공됩니다. 2015년에 설립된 Algoseek는 거래소에서 직접 기록된 우수한 특정 시점 데이터와 지연 시간이 짧은 실시간 피드를 제공합니다. 데이터 임대를 포함한 확장 가능한 가격 계획을 제공하여 퀀트가 더 낮은 가격으로 대규모 데이터셋에 접근할 수 있습니다.

대부분의 자산은 상장되었다가 결국 상장 폐지됩니다. QuantConnect에서 자산은 증권(qnt.co/book-securities)이라고 불리며 중앙 증권 컬렉션에 저장됩니다. 대부분의 자산 클래스는 시장 시간, 마진 요구사항, 손익 회계, 계약 승수와 같은 증권 속성과 같은 공통 속성을 공유합니다.

```python
self.security["SPY"] # 컬렉션에서 SPY 증권
self.security["SPY"].price # SPY의 현재 가격
self.security["SPY"].symbol # SPY 심볼 객체
self.security["SPY"].holdings.quantity # 보유 중인 SPY 주식 수
```

**미국 주식**

1817년에 설립된 미국 주식 시장은 세계에서 가장 크고 유동적인 시장입니다. 세계 시가총액의 절반을 차지하며 약 9,000개의 기업이 상장되어 있습니다.
기업공개(IPO), 정기 및 특별 배당, 주식 분할 및 역분할, 합병 및 인수 등과 같은 회사 수명 주기의 이벤트를 기업 행동이라고 합니다. 이러한 이벤트는 회사의 가치에 영향을 미치며 이해하는 것이 중요한 요소입니다. 다음 장에서는 기업 행동에 대해 훈련된 AI가 거래 전략에 어떻게 배치될 수 있는지에 대한 예를 살펴볼 것입니다.

수익, 비용, 재무제표에 보고된 수익성과 같은 기업 재무 성과 데이터—대차대조표, 손익계산서, 현금흐름표, 수익 예측 데이터—는 회사의 기본 데이터입니다. 기본 데이터 제공업체는 성실하게 회사의 분기별 SEC 제출을 기록합니다. 일부는 다른 회계 관행이나 관행을 가진 산업 및 시장 전반에서 비교할 수 있는 조정된 숫자로 보고된 항목을 표준화(또는 조화)합니다. 재무제표 데이터에서 재무 분석가는 주가수익(PE) 비율 및 자산 대비 자유현금흐름 비율과 같은 재무 비율을 공식화하여 다른 회사의 가치 평가를 비교하는 일반적인 지표로 사용합니다. 다음 장에서는 이러한 기본 비율을 기반으로 자산 가격을 예측하거나 거래 전략을 공식화하기 위해 기계 학습 기술을 적용하는 예를 살펴볼 것입니다.

QuantConnect에서는 개별 회사의 기본 데이터를 쉽게 가져오고 기본 데이터를 기반으로 주식 스크린을 개발할 수 있습니다.

```python
self.add_universe(self._filter) # 필터링된 주식 데이터 유니버스 추가
def _filter(self, fundamental): # pe_ratio를 기반으로 심볼 선택
    return [c.symbol for c in fundamental if c.valuation_ratios.pe_ratio < 10]
```

책 뒷부분에서 사용되는 일부 비율은 다음과 같습니다:

- PE 비율-자산의 종가를 주당 순이익으로 나눈 값.
- 수익 성장—손익계산서에서 회사 수익의 성장률(%).
- 자유 현금 흐름 비율—영업 현금 흐름의 비율로서의 자유 현금 흐름.
- 배당 지급 비율-순이익 대비 배당금 지급 비율.


**미국 주식 기업 이벤트**

회사 수명 주기 동안 IPO, 2차 공모, 현금 또는 주식 배당, 주식 분할 및 역분할, 티커 및 회사 이름 변경, 합병 및 인수, 마지막으로 상장 폐지와 같은 많은 동일한 기업 이벤트가 발생합니다. QuantConnect에서 이러한 이벤트는 다음과 같이 모델링됩니다:

분할-회사는 가격이 극적으로 변할 때 시장 참여자들이 더 쉽게 접근할 수 있도록 주식을 나누거나 통합할 수 있습니다(그림 1.10). 회사 가치가 성장함에 따라 주당 가격은 소매 투자자의 구매력을 초과할 수 있습니다. 고전적인 예는 버크셔 해서웨이 A클래스 주식으로, 한 번도 분할되지 않았으며 오늘날 주당 $\$ 200,000$ 이상의 가치가 있습니다. 이는 거래량이 낮아지고 잠재적으로 효율성이 떨어지는 시장을 초래합니다. 마찬가지로, 자산 가격이 하락하면 상장을 유지하기 위해 가격을 $\$ 1$ 이상으로 유지하도록 거래소의 압력을 받게 됩니다—역분할에서 주식(예: 두 개를 하나로 교환)을 통합함으로써 회사는 상장 가격을 두 배로 늘려 거래소 규칙을 준수할 수 있습니다.

<img src="./images/fig_01_10.png" width=800>

그림 1.10 분할 이벤트 전후의 주식 가치 및 보유 주식 수.

QuantConnect에서 분할은 거래를 위한 알고리즘 응답을 트리거하거나 연구를 위해 데이터 시리즈에 통합할 수 있는 분할 이벤트 핸들러에 전달되는 이벤트입니다:

```python
def on_splits(self, splits): # 전용 분할 이벤트 핸들러
    split = splits.get(self._symbol)
def on_data(self, slice: # on_data 이벤트의 분할
    split = slice.splits.get(self._symbol)
```

분할은 자산의 가격을 즉시 변경하므로, 연속적인 가격에 의존할 수 있는 지표와 같은 알고리즘 상태를 재설정하는 데 주의가 필요합니다.

    - **배당금**-회사가 수익성을 통과함에 따라 일부는 주주들에게 배당금을 발행하기로 선택합니다. 이러한 배당금은 일반적으로 분기별로 지급되지만, 궁극적으로 분배 금액과 빈도는 회사의 최고 재무 책임자와 이사회에 달려 있습니다. 배당금은 주당 달러로 인용되며 현금으로 귀하의 브로커리지에 분배됩니다. QuantConnect에서 이러한 배당금은 배당금 이벤트 핸들러에 전달되며 알고리즘 응답을 유도하는 데 사용될 수 있습니다:

```python
def on_data(self, slice):
    dividend = slice.dividends.get(self._symbol)
def on_dividends(self, dividends):
    dividend = dividends.get(self._symbol)
```

분할과 배당금 모두에 대한 역사적 데이터를 요청할 수 있습니다. 이는 자산의 배당 수익률을 수동으로 계산하거나 배당 추세를 예측하는 데 도움이 됩니다.

```python
# 지난 해에 지급된 심볼의 모든 배당금 가져오기
history = self.history[Dividend](symbols,timedelta(365))
```

지급된 배당금에서 받은 가치는 현금으로 지급으로 모델링되거나, 데이터 정규화라고 하는 과정을 통해 자산 가격에 적용될 수 있습니다.

    - **데이터 정규화**-두 증권을 정확하게 비교하기 위해, 우리는 데이터 정규화라고 하는 과정에서 가격 분할과 배당금 지급을 고려하기 위해 역사적 가격을 조정해야 합니다. 이러한 조정은 가치 성장을 가능한 한 정확하고 부드럽게 반영하기 위해 역사적 가격을 변경하는 공식을 사용합니다. 데이터 정규화에는 이 책에서 다루지 않을 문제가 있습니다. 그러나 첫 번째 패스로, 조정된 가격 데이터는 분할과 배당금을 통합하여 연구 과정을 단순화하는 좋은 일을 합니다. 배당금과 분할이 투자 전략에 미치는 영향에 대한 더 현실적인 회계를 위해, QuantConnect는 배당금을 포트폴리오에 현금으로 적용하고 언제 그리고 어떻게 재투자할지 결정할 수 있게 하는 원시 모드를 지원합니다.

QuantConnect에서는 자산을 전략에 추가하는 동안 데이터 정규화 모드를 선택합니다:

```python
# 역사적 가격의 조정 없음, 배당금은 현금으로 지급
self.add_equity("AAPL",data_normalization_mode=DataNormalizationMode.RAW)

# 역사적 가격의 전체 분할 및 배당금 조정
self.add_equity("AAPL", data_normalization_mode=DataNormalizationMode.ADJUSTED)

# 분할에 대해서만 조정하고, 배당금을 현금으로 지급
self.add_equity("AAPL", data_normalization_mode=DataNormalizationMode.SPLIT_ADJUSTED)
```

    - **증권 변경-IPO, 이름 변경, 분사, 합병 및 상장 폐지**-회사의 정규 활동은 퀀트에게 많은 모델링 과제를 생성합니다. 이러한 기업 이벤트는 가격에 영향을 미치지 않지만, 거래소에 상장 상태를 추적합니다. 포트폴리오 가치가 변화를 통해 연속적으로 유지되도록 하는 것이 중요합니다. 다음에서 이들 중 일부를 간략히 검토하고 QuantConnect에서 어떻게 처리되는지 보여드리겠습니다.

    IPO-회사가 준비되면, 거래소에 공개적으로 상장을 신청할 수 있습니다. 승인되면 IPO로 공식적으로 상장됩니다. 자산이 알고리즘에 추가될 때, 새로운 상장의 세부 정보와 함께 증권 변경 이벤트가 생성됩니다.

```python
def on_securities_changed(self, changes):
    for security in changes.added_securities:
        Pass
```

이름 변경-회사는 종종 새로운 사업 라인을 반영하거나 더 나은 시장 티커로 교체하기 위해 리브랜딩합니다. 이러한 티커 변경은 심볼 변경 이벤트에서 처리되며, 전략에 새로운 티커와 이전 티커를 제공합니다. QuantConnect는 자산을 추적하기 위해 심볼 객체를 사용하므로 새 티커로 전환할 필요가 없습니다.

```python
def on_symbol_changed_events(self, changes):
    for symbol, change in changes.items():
        self.log(f"Change: { change.old_symbol} ->{change.new_symbol}")
```

    - 분사 및 합병-회사는 때때로 작은 회사를 인수하여 거래에서 작은 회사를 흡수합니다. 이는 작은 회사의 단순한 상장 폐지로 모델링됩니다. 합병은 비슷하지만 새로운 티커를 초래할 수 있습니다; 이는 상장 폐지와 이름 변경으로 모델링됩니다.

    상장 폐지-대부분의 자산 클래스에는 일종의 상장 폐지가 있습니다. 주식의 경우, 이는 공개 시장에서 회사가 제거되는 것입니다. 이벤트는 상장 폐지 이벤트 핸들러에서 캡처됩니다. 또한, 제거된 증권으로 증권 변경 이벤트에 표시됩니다.

```python
def on_delistings(self, delistings): # 자산 상장 폐지 이벤트
    delisting = delistings.get(self._symbol)
def on_securities_changed(self, changes): # 선택된 스크린에서 제거
    for security in changes.removed_securities:
        self.log(f'Security {security.Symbol} removed from universe')
```


**미국 주식 옵션**

옵션은 특정 미래 날짜(만료일 또는 만기일이라고 함)에 특정 미리 결정된 가격(행사가라고 함)으로 자산을 구매할 권리는 있지만 의무는 없다는 계약입니다. 그들은 파생상품으로 분류되며, 옵션의 이론적 가격은 강하게

기초 자산에 의존합니다. 옵션 만료 시, 내가격 옵션은 기초 증권의 주식으로 교환됩니다.

다음은 미국 주식 옵션의 주요 속성입니다:

- 기초 자산—이것은 계약의 기초 주식으로, 기초 속성으로 접근할 수 있습니다.
- 옵션 권리-콜은 보유자에게 특정 가격에 주식을 살 권리를 주는 반면, 풋은 보유자에게 주식을 팔 권리를 줍니다.
- 행사가-행사가는 옵션의 계약 가격이며 옵션의 가치와 만료 시 어떻게 행사될지 결정하는 데 중요합니다.
- 만료일-옵션이 거래되거나 행사될 수 있는 마지막 날짜. 미국식 옵션은 언제든지 행사될 수 있으며 기초 주식의 물리적 인도로 결제됩니다.
- 계약 승수-주식 옵션에서, 각 옵션 계약은 기초 자산의 100주를 나타냅니다. 이 계약 승수는 실제 "청구서" 가격과 옵션 계약의 지급액을 계산하기 위해 상장된 가격에 곱해져야 합니다.

옵션 데이터는 미국 옵션 거래소의 가격을 집계하는 Options Price Reporting Authority (OPRA)에 의해 배포됩니다. 정기적으로 만료되는 계약과 행사가로 인해, 옵션은 주식보다 규모로 더 많은 가격 데이터를 생성합니다. 주어진 날에, 약 4,000개의 가장 유동적인 회사에 대해 약 150만 개의 옵션 계약이 사용 가능합니다. 이는 수백 테라바이트의 금융 데이터를 생성하며 처리하고 분석하기 어려울 수 있습니다.
QuantConnect에서, 옵션 데이터는 개별적으로 추가되거나 자산 유니버스에서 필터링될 수 있습니다. 각 기초 자산에는 수백 개의 옵션 계약이 있으며, 각 자산을 기반으로 유니버스를 형성합니다.

```python
option = self.add_option("SPY") # SPY에 대한 옵션 유니버스 요청
option.set_filter( ... ) # 반환된 옵션 계약 필터링

self.add_option_contract(contract_symbol) # 특정 옵션 계약
def on_data(self, slice): # on_data에서 옵션 체인 사용
    chain = slice.option_chains.get(option.symbol)
```

기초 주식 가격이 콜(풋) 옵션의 행사가보다 높을(낮을) 때, 옵션은 내가격(ITM)에 있다고 합니다. 즉, 행사되면, 옵션은 옵션 구매자에게 긍정적인 현금 흐름을 이끌 것입니다; 반대로 기초 주식 가격이 콜(풋) 옵션의 행사가보다 낮을(높을) 때, 옵션은 외가격(OTM)에 있다고 합니다. OTM 옵션은 가능성이 낮은 이벤트에 대한 보험 정책으로 사용될 수 있습니다.

대부분의 미국 주식 옵션은 물리적으로 결제됩니다—옵션 계약에 규정된 조건에 따라 주식의 이전을 통해. 미국 주식 옵션이 대부분 만료일 이전에 행사될 수 있는 "미국식" 옵션이지만, 콜 옵션이 조기에 행사될 가능성은 낮습니다. 옵션 가격 이론에 따르면 만료 전 콜 옵션의 가치는 옵션을 행사할 경우의 내재 가치보다 항상 높아야 하기 때문입니다. ITM 콜 옵션에 대한 이득을 확보하기 위해, 올바른 행동은 콜 옵션 포지션을 "매도하여 청산"하는 것입니다. ITM 콜 옵션을 만기까지 보유하면, 브로커가 자동으로 귀하를 대신하여 행사하며, 귀하는 기초 주식을 받고 행사가에 현금으로 지불하게 됩니다. 물론, 이 경우, 행사가가 기초의 시장 가격보다 낮기 때문에, 주식을 즉시 매도하여 이득을 얻을 수 있습니다(판매 시 세금 결과에 주의하세요).

반면에, 미국식 풋 옵션은 조기에 행사될 수 있으며 자주 행사됩니다. 풋 옵션을 판매하면, 구매자는 만료 전에 자신의 주식을 귀하에게 팔 권리를 행사할 수 있습니다. 이 경우, 귀하는 시장 가격보다 높은 가격에 주식을 구매하고 받게 되어 손실을 입게 됩니다. 보험사로서, 귀하는 청구를 지불할 책임이 있습니다. ITM 풋이 만료까지 보유된다면, 풋은 풋 구매자의 브로커에 의해 자동으로 행사될 것입니다. 조기 행사와 만료 행사에 대한 "할당" 과정은 옵션 거래소에 의해 관리되고 선입선출(FIFO) 기준으로 브로커에 의해 용이하게 됩니다.

**지수 옵션**

주식 지수에 대한 옵션은 두 가지 중요한 차이점을 제외하고 개별 주식 옵션과 유사하게 작동합니다: 그들은 "유럽식"입니다. 즉, 만료일에만 행사될 수 있으며, 현금으로 결제되며, 지급액은 지수 수준과 행사 수준의 차이에 계약 승수를 곱한 값으로 계산됩니다.

QuantConnect는 주간 버전과 함께 세 가지 지수 옵션 월간 옵션 체인을 지원합니다: VIX (VIXW), NDX (NQX), SPX (SPXW).

NDX—NASDAQ은 Nasdaq100 지수에 정규(NDX) 유럽식 옵션을 상장합니다. NDX의 참조 지수는 Nasdaq-100의 수준이며, 계약 승수는 100입니다.
SPX—시카고 옵션 거래소(CBOE)는 S\&P500 지수에 정규 SPX 유럽식 옵션을 상장합니다. SPX의 참조 지수는 S\&P500의 수준이며, 계약 승수는 100입니다.
VIX—CBOE는 S\&P500 지수의 30일 예상 변동성 측정을 추정하는 SPX 옵션 매수/매도 호가를 사용하여 계산된 정규 VIX 유럽식 옵션을 상장합니다. 계약 승수는 100입니다.

이러한 지수는 다음과 같이 알고리즘에 추가될 수 있습니다:

```python
self._index_symbol = self.add_index('SPX').symbol # 알고리즘에 지수 추가
option = self.add_index_option(self._index_symbol) # 지수에 옵션 추가
option.set_filter(-2, 2, 0, 90) # 계약 필터링
```

현금 결제로 인해, 미국 국세청은 지수 옵션과 주식 옵션에 대해 다른 세금 처리를 합니다. 자세한 내용은 세무 상담사에게 문의하십시오. 많은 브로커리지는 지수 옵션을 거래하기 위해 특별한 허가를 요구합니다. 일부는 다른 세금 처리로 인해 지수 옵션 거래를 허용하지 않습니다. 지수 옵션 거래에 대한 계정의 적격성을 결정하기 위해 브로커에게 문의하십시오.

**미국 선물**

미국 선물 시장은 원래 1850년대에 밀, 옥수수, 콩과 같은 농산물을 위해 만들어졌으며, 나중에 육류, 목재, 산업 및 귀금속, 에너지 자원, 채권, 지수, 통화와 같은 금융 상품을 포함한 다른 상품으로 확장되었습니다. 그들은 상품 생산자가 시장 위험을 줄이고 사전에 생산물에 대한 가격을 고정할 수 있게 하여, 생산자가 미래 수익의 상대적 확실성으로 생산을 계획할 수 있게 했습니다. 시카고 상품 거래소(CME) 그룹은 미국 최대의 선물 거래소 모음이며, Intercontinental Exchange (ICE) 그룹이 뒤를 이어 따라옵니다.

선물 계약에는 결제가 수행되는 만료일이 있습니다. 계약은 일반적으로 월간 또는 분기별 계약으로 분류됩니다. QuantConnect는 160개의 가장 유동적인 선물 상품을 지원합니다. 각 상품에는 개별 만료일을 나타내는 선물 계약 모음이 있으며, 유니버스를 형성합니다:

```python
future = self.add_future(Futures.Indices.SP_500_E_MINI)
# ES 선물 추가
future.set_filter(0, 90) # 90일 필터링
```

각 선물 상품("체인")에 대해, 서로 다른 만료일과 다른 가격으로 많은 중복된 선물 계약이 활성화되어 있습니다. 기초 상품이나 증권(현물이라고 함)을 가장 밀접하게 추적하는 선물 계약을 최근월물이라고 합니다. "롤링"은 다음 최근월물 계약을 선택하는 과정입니다. 이 롤이 발생하면, 최근월물 계약의 가격이 점프합니다. 왜냐하면 만료일이 다른 두 개의 다른 계약이기 때문입니다.

분석을 위한 최근 계약의 정규화된 가격 시리즈를 생성하기 위해, 많은 계약을 함께 이어 가격 시리즈를 형성하는 가상 연속 최근 계약이 생성되었습니다. 참고로, 연속 최근 계약 가격은 기초 계약의 실제 가격이 아니라, 데이터 분석을 위한 기본 가치의 연속적인 움직임을 포착하기 위한 정규화된 가격 시리즈입니다. 이 정규화된 시리즈를 생성하는 많은 방법이 있지만, 가격 순간의 표현일 뿐이므로 하나의 "올바른"

방법은 없습니다. QuantConnect에서는 add_future 메서드로 지정할 수 있는 9가지 정규화 방법 조합이 가능합니다.

QuantConnect에서, 이 연속 최근 계약은 표준 계약으로 참조됩니다. 현재 기초, 거래 가능한 계약은 맵핑된 자산이라고 불립니다. 이는 다음과 같이 액세스할 수 있습니다:

```python
future = self.add_future(Futures.Indices.SP_500_E_MINI) # 표준 자산
adjusted_price = self.security [future.symbol].price # 조정된 가격
raw_price = self.security[future.mapped].price
# 원시 가격
```

거래를 라우팅할 때, 연속 계약은 거래할 수 없으므로 기초 맵핑된 자산을 사용해야 합니다.

```python
self.market_order(future.mapped, 1) # 맵핑된 자산의 1 계약 구매
```

실시간 거래에서 최근 계약이 롤될 때, 맵핑된 자산의 연속 가격에 불연속성이 있을 것입니다. 따라서, 연속 "표준" 가격에 구축된 모든 지표는 롤 후 맵핑된 자산 가격의 점프를 조정하기 위해 검토되고 재설정되어야 합니다. 결제 후, 거래소는 계약의 상장을 폐지하여 QuantConnect에서 상장 폐지 이벤트를 트리거합니다.

대부분의 현대 선물 시장은 이익과 손실을 매일 현금으로 결제하므로, 만료 시 큰 현금 지급은 없습니다. 그러나 대부분의 상품 및 금융 선물은 여전히 현물로 결제되어 물리적 자산의 인도가 필요합니다. 따라서, 일반적으로 인도를 받거나 만들 의도가 없는 투기적 트레이더는 만료 전에 선물 포지션을 닫거나 롤해야 합니다. 많은 브로커리지는 물리적 인도 선물이 만료에 가까워질 때 자동 청산 정책을 시행합니다. 이는 금융 브로커가 지원하지 않는 인도의 위험을 제거합니다. 이러한 청산은 투자 포트폴리오에 의도하지 않은, 그리고 아마도 부정적인 결과를 초래할 수 있으므로, 트레이더는 강제 청산 전에 선물 포지션을 닫거나 롤하기 위해 조기 청산 날짜에 대해 브로커와 상담하는 것이 좋습니다.

**암호화폐**

지난 10년 동안, 암호화폐는 새로운 글로벌 자산 클래스로 폭발적으로 성장했습니다. 분산화된 가치 저장소이자 비즈니스 생태계로서, 그것은 폭넓은 매력을 가졌습니다. 초기 채택자들은 엄청난 가치 상승으로 잘 보상받았습니다.

거래는 중앙화된 거래소와 분산화된 거래소(DEX) 사이에 분할됩니다. 대부분의 거래소는 현대적인 애플리케이션 프로그램 인터페이스(API)를 갖춘 기술자들에 의해 구축되었지만, 전통적인 금융 시장을 뒷받침하는 금융 인프라에 대한 지식은 제한적이었습니다. 그렇기 때문에, 그들은 SIP가 전국적인 피드로 연결하기 전에 연결이 끊기고 독립적이었던 20세기 중반 미국 거래소와 유사한 방식으로 운영됩니다. 대부분은 유동성을 제공하기 위해 사설 마켓 메이커에 의존하고, 거래소 간 자산 가격을 균형잡기 위해 거래소 간 차익 거래를 수행하는 다른 펀드들에 의존합니다.

그러나, 암호화폐 거래 인프라는 전통적인 금융 시장의 인프라를 빠르게 따라잡고 있습니다. 첫 번째 암호화폐 거래소는 현금 계좌에서만 현물 가격 거래로 시작되었습니다. 이제 마진 거래를 제공하고, 연속 선물이나 옵션과 유사한 암호화폐 파생상품에 대한 거래를 제공하는 수백 개의 거래소가 있습니다.

서로 다른 가격을 가지고 있고 중앙화된 피드가 없기 때문에, 암호화폐 거래소는 다른 시장으로 간주됩니다. 이들은 add crypto API로 추가할 수 있습니다:

```python
# 시장별로 두 거래소의 가격 피드 요청
coinbase_btcusd = self.add_crypto("BTCUSD",market=Market.COINBASE).symbol
kraken_btcusd = self.add_crypto("BTCUSD",market=Market.KRAKEN).symbol
```

각 거래소는 지원되는 마진, 자산, 주문 유형이 다르므로, 관련 거래소에 대한 브로커리지 모델을 설정하는 것이 중요합니다. 여기서 계정 유형(현금 또는 마진)도 설정할 수 있습니다.

```python
self.set_brokerage_model(BrokerageName.KRAKEN, AccountType.MARGIN)
```

---